# Poland 2D Line 001 drop auxiliary channels
This example segy file has auxiliary channels, i.e. traces that do not map to the field acquisition geometry, but intended for other purposes. To drop those traces, this example creates a new bytearray object and assigns byte data if the trace number within ensemble (often referred to as "channel") is greater than zero. The aux channels in this segy file have a value of -1.

This example uses the open source 2D land vibroseis line from [SEG Wiki](https://wiki.seg.org/wiki/2D_Vibroseis_Line_001).

In [1]:
import os
from pathlib import Path
from poland2d_context import segytools

TEXTENCODE = 'ebcdic'
BYTEORDER = '>'
TEXTHEADERLENGTH = 3200
FILEHEADERLENGTH = 400
TRCHEADERLENGTH = 240

segyfile = "data/Line_001.sgy"
pathsegyfile = Path(str(segyfile))
assert (pathsegyfile.is_file())

segyfilesize = os.path.getsize(segyfile)

# Initialize an empty bytearray that is the same size as the input segy file.
# This is only possible with small files where segyfilesize < available memory.
b_sgy_arr = bytearray(segyfilesize)

file_header = segytools.SegyFileHeaderRev2()
trace_header = segytools.SegyTraceHeaderRev2()

# 'rb' is "read bytes"
with open(segyfile, 'rb') as fobj:
    # read the first 3200 bytes.
    # This will always be 3200 byte textual file header
    b_text_header = fobj.read(TEXTHEADERLENGTH)
    b_sgy_arr[0:TEXTHEADERLENGTH] = b_text_header
    last_byte = fobj.tell()

    b_file_header = fobj.read(FILEHEADERLENGTH)
    file_header.set_header_values(buf=b_file_header, byteorder=BYTEORDER)
    # modify file header here ...
    b_sgy_arr[last_byte:last_byte+FILEHEADERLENGTH] = file_header.to_bytes(byteorder=BYTEORDER)
    last_byte = fobj.tell()
    
    sample_size_in_bytes = file_header.sample_format_size_in_bytes()
    trc_data_length_in_bytes = file_header.num_samples_per_trace.value * sample_size_in_bytes
    
    # Loop through traces ...
    while fobj.tell() < segyfilesize:
        # TRACE HEADER
        b_trace_header = fobj.read(TRCHEADERLENGTH)
        trace_header.set_header_values(buf=b_trace_header, byteorder=BYTEORDER)
        if trace_header.num_samples.value != file_header.num_samples_per_trace.value:
            trc_data_length_in_bytes = file_header.num_samples_per_trace.value * sample_size_in_bytes
        # write trace header to bytearray `b_sgy_arr`
        if trace_header.trc_num_within_field_record.value > 0:
            b_sgy_arr[last_byte:last_byte+TRCHEADERLENGTH] = trace_header.to_bytes(byteorder=BYTEORDER)
            last_byte += TRCHEADERLENGTH

        # TRACE DATA
        b_trace_data = fobj.read(trc_data_length_in_bytes)
        if trace_header.trc_num_within_field_record.value > 0:
            b_sgy_arr[last_byte:last_byte+trc_data_length_in_bytes] = b_trace_data
            last_byte += trc_data_length_in_bytes

    fobj.close()

# write output segy; use only with small segy files
segy_file_out = os.path.join(os.getcwd(), "data", "Line_001_NO_AUX.sgy")
with open(segy_file_out, "wb") as fobj2:
    bsgyout = bytes(b_sgy_arr[:last_byte])
    fobj2.write(bsgyout)
    fobj2.close()    